In [ ]:
from __future__ import annotations

import legendhpges as hpges
import pyg4ometry as pg4
import numpy as np
from pygeomtools import viewer
import pygeomtools

reg = pg4.geant4.Registry()

# create a world volume
world_s = pg4.geant4.solid.Orb("world", 150, registry=reg, lunit="mm")
world_l = pg4.geant4.LogicalVolume(world_s, "G4_Galactic", "world", registry=reg)
reg.setWorld(world_l)




# let's make a germanium balloon
natge = hpges.materials.make_natural_germanium(reg)
ge_s = pg4.geant4.solid.Tubs(
    "germanium",
    pRMin=0,
    pRMax=10,
    pDz=10,
    pSPhi=0,
    pDPhi=2 * np.pi,
    registry=reg,
    lunit="mm",
)
ge_l = pg4.geant4.LogicalVolume(ge_s, natge, "germanium", registry=reg)
pg4.geant4.PhysicalVolume([0, 0, 0], [0, 0, 0], ge_l, "germanium", world_l, registry=reg)
ge_l.pygeom_color_rgba = (0, 1, 1, 0.9)

shield_s = pg4.geant4.solid.Tubs("Al_s", 0, 10, 0.5, 0, 2 * np.pi, registry=reg)
shield_l = pg4.geant4.LogicalVolume(shield_s, "G4_Al", "Shield_L", registry=reg)
pg4.geant4.PhysicalVolume(
    [0, 0, 0], [0, 0, 0.8, "cm"], shield_l, "Shield", world_l, registry=reg
)

shield_l.pygeom_color_rgba = (0, 1, 0, 0.9)

# finally create a small radioactive source
source_s = pg4.geant4.solid.Tubs("Source_s", 0, 1, 1, 0, 2 * np.pi, registry=reg)
source_l = pg4.geant4.LogicalVolume(source_s, "G4_Fe", "Source_L", registry=reg)
pg4.geant4.PhysicalVolume(
    [0, 0, 0], [0, 0, 1.5, "cm"], source_l, "Source", world_l, registry=reg
)

viewer.visualize(reg)
pygeomtools.visualization.write_color_auxvals(reg)

w = pg4.gdml.Writer()
w.addDetector(reg)

w.write("gdml/geometry_fccd.gdml")